In [1]:
from PIL import Image
from PIL.TiffTags import TAGS
import re
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.transform import Affine
import numpy as np 
import cv2
import imutils
import matplotlib.pyplot as plt

In [3]:
def metadata(imagen):
    img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
    meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}
    # Se imprime el diccionario para obtener la composición de los datos
    # for rec in meta_dict:
    #     print(rec, ":", meta_dict[rec])

    # Extraemos el indicador xmp
    p = meta_dict.get("XMP")
    s = p.decode("UTF-8")
    # dividimos por el salto de linea y obtenemos una lista
    div = s.split("\n")

    #eliminamos los espacios vacios 

    for ind,recorrido in enumerate(div):
        div[ind]=recorrido.strip() 
    usar = div[17]

    result = re.search(":(.*)=", div[17])
    result.group(1)

    result2 = re.search("\"(.*)\"", div[17])
    result2.group(1)

    metadiccionario = {}
    for ind,recorrido in enumerate(div):
        try:
            metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
        except:
            pass

    metadiccionario = {}
    for ind,recorrido in enumerate(div):
        try:
            metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
        except:
            pass

    print(metadiccionario)
    return metadiccionario

In [5]:
img_path = "Img/DJI_0011.TIF"  #Dirrecion de la imagen .TIF que se va a rotar

metadiccionario = metadata(img_path)

centro_x = float(metadiccionario["CalibratedOpticalCenterX"])
centro_y = float(metadiccionario["CalibratedOpticalCenterY"])
altura_vuelo = float(metadiccionario["RelativeAltitude"])
distancia_focal = float(metadiccionario["CalibratedFocalLength"])
angulo_giro = float(metadiccionario["GimbalYawDegree"])
latitud = float(metadiccionario["GpsLatitude"])
longitud = float(metadiccionario["GpsLongitude"])

resolucion = altura_vuelo/distancia_focal
print(f"Resolución = {resolucion}")

min_lon = float(metadiccionario["GpsLongitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
max_lon = float(metadiccionario["GpsLongitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
min_lat = float(metadiccionario["GpsLatitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
max_lat = float(metadiccionario["GpsLatitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
print(min_lon, max_lon, min_lat, max_lat)


img_data = rasterio.open(img_path, 'r')

data = img_data.read()
_, width, height = data.shape
transform = rasterio.transform.from_bounds(
    min_lon, min_lat, max_lon, max_lat, width, height)

crs = {'init': 'epsg:4326'}

print(data.shape)

{'xmpmeta xmlns:x': 'adobe:ns:meta/', 'RDF xmlns:rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'Description rdf:about': 'DJI Meta Data', 'tiff': 'http://ns.adobe.com/tiff/1.0/', 'exif': 'http://ns.adobe.com/exif/1.0/', 'xmp': 'http://ns.adobe.com/xap/1.0/', 'xmpMM': 'http://ns.adobe.com/xap/1.0/mm/', 'dc': 'http://purl.org/dc/elements/1.1/', 'crs': 'http://ns.adobe.com/camera-raw-settings/1.0/', 'drone-dji': 'http://www.dji.com/drone-dji/1.0/', 'Camera': 'http://pix4d.com/camera/1.0', 'ModifyDate': '2022-02-11', 'CreateDate': '2022-02-11', 'Make': 'DJI', 'Model': 'FC6360', 'format': 'image/TIF', 'AbsoluteAltitude': '+84.92', 'RelativeAltitude': '+70.20', 'GpsLatitude': '11.22621616', 'GpsLongitude': '-74.18506225', 'GimbalRollDegree': '+0.00', 'GimbalYawDegree': '-95.70', 'GimbalPitchDegree': '-89.90', 'FlightRollDegree': '+6.50', 'FlightYawDegree': '-94.90', 'FlightPitchDegree': '+11.00', 'FlightXSpeed': '+0.00', 'FlightYSpeed': '+0.10', 'FlightZSpeed': '+0.00', 'CamReverse': '

In [7]:
with rasterio.open(
        "Img_rotado/Img_rot12.TIF",  # Nombre del nuevo archivo georefenciado
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype="float64",
        crs=crs,
        transform=transform,
    ) as dst:
    dst.write(data)

AttributeError: 'NoneType' object has no attribute 'shape'